In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\Users\\liamk\\Documents\\spark\\spark-3.3.1-bin-hadoop3'

In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

In [4]:
spark = SparkSession.builder.appName('ml').getOrCreate()

In [5]:
path = '../Datasets/cogsley_sales.csv'

data = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load(path)

In [6]:
data.cache() # cache data for faster reuse
data = data.dropna()

In [7]:
data.head()

Row(RowID=1, OrderID=3, OrderDate=datetime.datetime(2010, 10, 13, 0, 0), OrderMonthYear=datetime.datetime(2010, 10, 1, 0, 0), Quantity=6, Quote=1200, DiscountPct=0.04, Rate=200, SaleAmount=1152.0, CustomerName='Muhammed MacIntyre', CompanyName='CA Inc.', Sector='Technology', Industry='Computer Software: Prepackaged Software', City='Highland Park', ZipCode=60035, State='Illinois', Region='Central', ProjectCompleteDate=datetime.datetime(2010, 10, 20, 0, 0), DaystoComplete=7, ProductKey='Development - Big Data', ProductCategory='Development', ProductSubCategory='Big Data', Consultant='Ethan Bird', Manager='Josh Martinez', HourlyWage=60, RowCount=1, WageMargin=0.7)

In [8]:
for field in data.schema.fields:
    print(field.name +" , "+str(field.dataType))

RowID , IntegerType()
OrderID , IntegerType()
OrderDate , TimestampType()
OrderMonthYear , TimestampType()
Quantity , IntegerType()
Quote , IntegerType()
DiscountPct , DoubleType()
Rate , IntegerType()
SaleAmount , DoubleType()
CustomerName , StringType()
CompanyName , StringType()
Sector , StringType()
Industry , StringType()
City , StringType()
ZipCode , IntegerType()
State , StringType()
Region , StringType()
ProjectCompleteDate , TimestampType()
DaystoComplete , IntegerType()
ProductKey , StringType()
ProductCategory , StringType()
ProductSubCategory , StringType()
Consultant , StringType()
Manager , StringType()
HourlyWage , IntegerType()
RowCount , IntegerType()
WageMargin , DoubleType()


In [9]:
summary = data.select("OrderMonthYear", "SaleAmount").groupby("OrderMonthYear").sum()\
    .orderBy("OrderMonthYear").toDF("OrderMonthYear", "SaleAmount")

results = summary.rdd.map(lambda r: (int(str(r.OrderMonthYear).replace('-', '')[:8]), r.SaleAmount))\
    .toDF(["OrderMonthYear", "SaleAmount"])

results.head()

Row(OrderMonthYear=20090101, SaleAmount=741024.2000000001)

In [10]:
dat = results.select("OrderMonthYear", "SaleAmount")\
    .rdd.map(lambda r: LabeledPoint(r[1], [r[0]]))\
    .toDF()

dat.head()

Row(features=DenseVector([20090101.0]), label=741024.2000000001)

## Fit linear models

In [14]:
lr = LinearRegression()

modelA = lr.fit(dat, {lr.regParam:0.0})
modelB = lr.fit(dat, {lr.regParam:100.0})

predA = modelA.transform(dat)
predB = modelB.transform(dat)

display(predA)

IllegalArgumentException: requirement failed: Column features must be of type class org.apache.spark.ml.linalg.VectorUDT:struct<type:tinyint,size:int,indices:array<int>,values:array<double>> but was actually class org.apache.spark.mllib.linalg.VectorUDT:struct<type:tinyint,size:int,indices:array<int>,values:array<double>>.

In [ ]:
spark.stop()

In [ ]:
import pyspark
import random
sc = pyspark.SparkContext(appName="Pi")
num_samples = 100000000
def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1
count = sc.parallelize(range(0, num_samples)).filter(inside).count()
pi = 4 * count / num_samples
print(pi)
sc.stop()

In [ ]:
sc.stop()